In [21]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_covtype
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import RUSBoostClassifier as RUS
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [22]:
!pip install ucimlrepo
!pip install category_encoders
!pip install utils

In [23]:
from ucimlrepo import fetch_ucirepo
from sklearn.datasets import fetch_openml
import category_encoders as ce

from itertools import combinations
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from utils import *
from tqdm import tqdm

In [34]:
dataset_list_openML = [
    fetch_openml(data_id=329), #Hayes-Roth
    fetch_openml(data_id=1523), #Vertebra-column
    fetch_openml(data_id=40682), #Thyroid-new
    fetch_openml(data_id=30), #Page-blocks
    fetch_openml(data_id=40672) #FARS (Big dataset)
]

dataset_name_list_openML = ['Hayes-Roth',
                     'Vertebral column',
                     'Thyroid',
                     'Page blocks',
                      'FARS(Big)']

dataset_list_uci = [
    fetch_ucirepo(id=109), #Wine dataset
    fetch_ucirepo(id=30), #contraceptive_method_choice dataset
    fetch_ucirepo(id=81), #Pen based dataset
    fetch_ucirepo(id=33), #Dermatology dataset
    fetch_ucirepo(id=12), #Balance scale dataset
    fetch_ucirepo(id=42), #Glass
    fetch_ucirepo(id=45), #Heart
    fetch_ucirepo(id=19), #Car evaluation
    fetch_ucirepo(id=110), #Yeast
    fetch_ucirepo(id=148), #Shuttle
]
dataset_name_list_uci = ['Wine',
                     'Contraceptive',
                     'Pen based',
                     'Dermatology',
                     'Balance scale',
                     'Glass','Heart',
                     'Car evaluation',
                     'Yeast',
                     'Shuttle']

dataset_list = dataset_list_uci+dataset_list_openML
dataset_name_list = dataset_name_list_uci+dataset_name_list_openML

In [35]:
len(dataset_list_openML),len(dataset_list_uci),len(dataset_list)

(5, 10, 15)

In [36]:
mapping_for_car_dataset = [{'col':'buying', 'mapping':{'low':0, 'med':1, 'high':2, 'vhigh':3}},
          {'col':'maint', 'mapping':{'low':0, 'med':1, 'high':2, 'vhigh':3}},
          {'col':'doors', 'mapping':{'2':0, '3':1, '4':2, '5more':3}},
          {'col':'persons', 'mapping':{'2':0, '4':1, 'more':2}},
          {'col':'lug_boot', 'mapping':{'small':0, 'med':1, 'big':2}},
          {'col':'safety', 'mapping':{'low':0, 'med':1, 'high':2}}]

In [49]:
def f_measure(y_true, y_pred, beta=1):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f_measure = (1 + beta) * (precision * recall) / (beta * precision + recall)
    return f_measure

def mmcc(y_true, y_pred, classes):
    mcc_values = []
    for class_pair in combinations(classes, 2):
        # Create binary arrays for each class in the pair
        y_true_binary = np.isin(y_true, class_pair)
        y_pred_binary = np.isin(y_pred, class_pair)
        mcc = matthews_corrcoef(y_true_binary, y_pred_binary)
        mcc_values.append(mcc)
    mmcc_value = np.mean(mcc_values)
    return mmcc_value


def macro_averaged_auprc(y_true, y_scores, n_classes):
    # Ensure y_true is binarized for multi-class labels
    y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))

    # Initialize list to store AUC scores for each class
    auc_scores = []

    # Compute Precision-Recall curve and AUC for each class
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_scores[:, i])
        auc_score = auc(recall, precision)
        auc_scores.append(auc_score)

    # Calculate the macro-averaged AUPRC
    macro_auprc = np.mean(auc_scores)
    return macro_auprc
def g_mean_multiclass(y_true, y_pred, n_classes):
    """
    Compute the G-mean for multi-class classification.

    Parameters:
    - y_true: array-like of shape (n_samples,), True labels for each sample.
    - y_pred: array-like of shape (n_samples,), Predicted labels for each sample.
    - n_classes: int, The number of unique classes.

    Returns:
    - g_mean: The geometric mean of recall for all classes.
    """
    recalls = []
    for i in range(n_classes):
        recall = recall_score(y_true, y_pred, labels=[i], average='weighted')
        recalls.append(recall)
    g_mean = np.sqrt(np.prod(recalls))
    return g_mean,recalls

In [50]:
clf_dict = {
    'AdaBoost' : AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42), # AdaBoost that use with SMOTE
    'RUS' : RUS(n_estimators=100, algorithm='SAMME.R', learning_rate=0.1 , random_state=42)
}
clf_score_dict = {}

In [51]:
def metric_list(y_test,y_preds,y_scores,num_of_classes):
    accuracy = accuracy_score(y_test, y_preds)
    precision = precision_score(y_test, y_preds, average='weighted')
    recall = recall_score(y_test, y_preds, average='weighted')
    f1 = f1_score(y_test, y_preds, average='weighted')
    Macro_Averaged =macro_averaged_auprc(y_true=y_test,y_scores=y_scores,n_classes=num_of_classes)
    F_measure = f_measure(y_test,y_preds,beta=10)
    Modified_mcc = mmcc(y_true=y_test,y_pred=y_preds,classes=range(num_of_classes))
    MCC_metric = matthews_corrcoef(y_test,y_preds)
    Gmean = np.prod(g_mean_multiclass(y_true=y_test,y_pred=y_preds,n_classes=num_of_classes)[1][1::])

    score_list = [accuracy,precision,recall,f1,Macro_Averaged,F_measure,Modified_mcc,MCC_metric,Gmean]
    return score_list

In [52]:
for clf_name, clf in tqdm(clf_dict.items()):
    clf_scores = {}
    for index, data in enumerate(dataset_list):

        # Data preparation
        if index in range(10):
            table = pd.concat([data.data.features, data.data.targets], axis=1)
        elif index in range(10, 15):
            table = pd.concat([data.data, data.target], axis=1)
        table = table.dropna()
        x = table.iloc[:, :-1]

        if index == 14:  # for FARS dataset
            categorical_columns = x.select_dtypes(include=['category']).columns
            label_encoder = LabelEncoder()
            for col in categorical_columns:
              x[col] = label_encoder.fit_transform(x[col])

        if index == 7:  # for Car evaluation dataset
            encoder = ce.OrdinalEncoder(cols=x.columns.values.tolist(), mapping=mapping_for_car_dataset)
            x = encoder.fit_transform(x)

        y = np.ravel(table.iloc[:, -1:])
        num_of_classes = np.unique(y).size
        print('\t Dataset : ', dataset_name_list[index], ' is processing')

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

        # Apply SMOTE if AdaBoost
        if clf_name == 'AdaBoost':
            smote = SMOTE(random_state=42, k_neighbors =3) # Set k_neighbors =3 because in Yeast dataset wil have error like n_neighbors <= n_samples,  but n_samples = 5, n_neighbors = 6
            X_train, y_train = smote.fit_resample(X_train, y_train)

        # Define and train classifier
        classification = clf
        classification.fit(X_train, y_train)

        # Predict and calculate scores
        y_preds = classification.predict(X_test)
        y_scores = classification.predict_proba(X_test)
        score_p = metric_list(y_test, y_preds, y_scores, num_of_classes)
        clf_scores[dataset_name_list[index]] = score_p

    # Convert scores to DataFrame
    df = pd.DataFrame.from_dict(clf_scores, orient='index', columns=['accuracy', 'precision', 'recall', 'f1', 'Macro-Averaged AUPRC',
                                                                    'F_measure Beta=10', 'Modified mcc', 'MCC metirc', 'Gmean'])
    clf_score_dict[clf_name] = df


  0%|          | 0/2 [00:00<?, ?it/s]

	 Dataset :  Wine  is processing
	 Dataset :  Contraceptive  is processing
	 Dataset :  Pen based  is processing
	 Dataset :  Dermatology  is processing
	 Dataset :  Balance scale  is processing
	 Dataset :  Glass  is processing
	 Dataset :  Heart  is processing
	 Dataset :  Car evaluation  is processing
	 Dataset :  Yeast  is processing
	 Dataset :  Shuttle  is processing
	 Dataset :  Hayes-Roth  is processing
	 Dataset :  Vertebral column  is processing
	 Dataset :  Thyroid  is processing
	 Dataset :  Page blocks  is processing
	 Dataset :  FARS(Big)  is processing


 50%|█████     | 1/2 [02:02<02:02, 122.97s/it]

	 Dataset :  Wine  is processing
	 Dataset :  Contraceptive  is processing
	 Dataset :  Pen based  is processing
	 Dataset :  Dermatology  is processing
	 Dataset :  Balance scale  is processing
	 Dataset :  Glass  is processing
	 Dataset :  Heart  is processing
	 Dataset :  Car evaluation  is processing
	 Dataset :  Yeast  is processing
	 Dataset :  Shuttle  is processing
	 Dataset :  Hayes-Roth  is processing
	 Dataset :  Vertebral column  is processing
	 Dataset :  Thyroid  is processing
	 Dataset :  Page blocks  is processing
	 Dataset :  FARS(Big)  is processing


100%|██████████| 2/2 [02:54<00:00, 87.20s/it]


In [44]:
clf_score_dict['AdaBoost'] # SMOTE that use with AdaBoost

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Wine,0.916667,0.931373,0.916667,0.917636,0.379154,0.917984,0.882965,0.877980,0.857143
Contraceptive,0.552542,0.617578,0.552542,0.559329,0.335579,0.557883,0.351982,0.348924,0.275406
Pen based,0.653024,0.645358,0.653024,0.620737,0.651934,0.652320,0.619726,0.621426,0.018766
Dermatology,0.763889,0.837218,0.763889,0.755116,0.260745,0.770020,0.757700,0.734161,0.361781
Balance scale,0.552000,0.926448,0.552000,0.629347,0.500000,0.573056,0.000000,0.502438,0.000000
Glass,0.558140,0.514886,0.558140,0.521890,0.320273,0.553909,0.248621,0.430765,0.000000
Heart,0.450000,0.601984,0.450000,0.502361,0.273784,0.460571,0.212582,0.215981,0.000000
Car evaluation,0.797688,0.857553,0.797688,0.815010,0.500000,0.802783,0.000000,0.644949,0.000000
Yeast,0.336700,0.474221,0.336700,0.327084,0.500000,0.345817,0.000000,0.200771,0.000000
Shuttle,0.561638,0.900934,0.561638,0.678529,0.352669,0.581548,0.334204,0.342837,0.016300


In [45]:
clf_score_dict['RUS']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Wine,0.944444,0.951389,0.944444,0.942328,0.378808,0.945072,0.909514,0.917649,1.000000
Contraceptive,0.555932,0.604383,0.555932,0.561156,0.356255,0.560013,0.348569,0.346517,0.270423
Pen based,0.657572,0.720474,0.657572,0.625359,0.671524,0.662833,0.627678,0.627787,0.008437
Dermatology,0.958333,0.963889,0.958333,0.959458,0.230054,0.958836,0.945586,0.948221,0.823052
Balance scale,0.688000,0.917222,0.688000,0.753453,0.500000,0.703994,0.000000,0.605646,0.000000
Glass,0.837209,0.846651,0.837209,0.836530,0.302588,0.838059,0.658141,0.793153,0.000000
Heart,0.500000,0.595518,0.500000,0.536004,0.288282,0.507399,0.227282,0.246515,0.000000
Car evaluation,0.791908,0.853743,0.791908,0.809551,0.500000,0.797156,0.000000,0.635515,0.000000
Yeast,0.511785,0.619925,0.511785,0.527643,0.500000,0.520031,0.000000,0.401078,0.000000
Shuttle,0.696293,0.926964,0.696293,0.791329,0.293680,0.712409,0.427818,0.464212,0.012718
